# Tutorial. The Fourier interpolation

Spectral methods are a class of spatial discretisation methods for differential equations in which the approximation of the solution $u$ of the problem is based an expansion in terms of so-called *trial functions* $\{\phi_k\}_{k=0,\dots,N-1}$,
$$
u(x)\approx\sum_{k=0}^{N-1}\tilde{u}_k\phi_k(x),
$$
the $N$ coefficients of the expansion being noted by $\tilde{u}_k$, $k=0,\dots,N-1$.

The choice of the trial function is dictated by the practical and computational efficiency of of the numerical method, and it has to meet the following requirements:
* *Convergence:* the approximation should converge rapidly to the solution $u$ as $N$ tends to $+\infty$,
* *Transformation:* the computation of the coefficients $\tilde{u}_k$ from the values of $u$ and the reconstruction of the function values at given nodes from the set of coefficients should be computationally fast,
* *Differentiation:* given the expansion coefficients of a function, it should be easy to determine the set of coefficients associated with an approximation of a spatial derivative of the function.

For periodic problems, the Fourier system, formed by the orthogonal (with respect to the $L^2((0,2\pi),\mathbb{C})$-scalar product) set of functions
$$
\forall k\in\mathbb{Z},\ \phi_k(x)=e^{ikx}
$$
over the interval $(0,2\pi)$ is an appropriate choice, due to the approximation properties of the Fourier series and the availability of a rapid algorithm, the [fast Fourier transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform), to compute the [discrete Fourier transform](https://en.wikipedia.org/wiki/Discrete_Fourier_transform) of a sequence. The interpolant of $u$ then takes the form
$$
\sum_{k=-K}^K\tilde{u}_k\phi_k
$$
if $N=2K+1$ is odd, or
$$
\sum_{k=-K+1}^K\tilde{u}_k\phi_k
$$
if $N=2K$ is even, where the coefficients $\tilde{u}_k$ are the *discrete Fourier coefficients* of the function $u$.

The present notebook aims at investigating some computational and numerical aspects of the Fourier interpolation of a periodic function. 

The <tt>numpy</tt> and <tt>matplotlib</tt> packages will be needed.

In [ ]:
import numpy as np

# To draw matplotlib plots within this notebook.
%matplotlib inline
import matplotlib.pyplot as plt

## Exercise 1. The Fourier interpolation using the fast Fourier transform.

Consider an *even* non-zero natural integer $N$ and the set of $N$ equispaced points in the interval (chosen for convenience) $[0,2\pi]$
$$
x_j=\frac{2\pi j}{N},\ j=0,\dots,N-1.
$$
Given a periodic function $u$, the *$\frac{N}{2}$-degree trigonometric interpolant of $u$ at the nodes $x_j$*, $j=0,\dots,N-1$, is
$$
I_Nu(x)=\frac{1}{N}\sum_{k=-N/2+1}^{N/2}\tilde{u}_k\,e^{ikx},
$$
the discrete Fourier coefficients in the expansion being given by
$$
\tilde{u}_k=\sum_{j=0}^{N-1}u(x_j)\,e^{-ikx_j},\ k=-\frac{N}{2}+1,\dots,\frac{N}{2}.
$$
The determination of the interpolant is directly related to the computation of its coefficients, which can be accomplished by the [fast Fourier transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) (FFT) algorithm, available in the `fft` library of <tt>numpy</tt> (see the [documentation](https://numpy.org/doc/stable/reference/routines.fft.html)). The computational complexity of this algorithm is $O(N\log_2(N))$.

**Question.** Write a function computing the expansion coefficients of the interpolant $I_Nu$ of a function $u$, the function and the integer $N$ being given. The complexity of this function should be a $O(N\log_2(N))$.

In [ ]:
def fourier_interpolant_coefficients(u,N):
    x=np.linspace(0,2*np.pi,N,endpoint=False)
    return np.fft.fft(u(x))

**Question.** Write a function which evaluates the interpolant $I_Nu$ of a function $u$ at a given set of points, the set of discrete Fourier coefficients being given.

In [ ]:
def evaluate_fourier_interpolant(coefficients,points):
    N=coefficients.size
    weights=1j*np.zeros(N) # or np.zeros(N,dtype='complex')
    values=1j*np.zeros(points.size)
    for i in range(points.size):
        if N%2==0: # the number of points is even: N=2K
            # K+1 coefficients with non negative index
            weights[0:N//2+1]=np.exp(1j*np.arange(0,N//2+1)*points[i])
            # K-1 coefficients with negative index
            weights[N//2+1:]=np.exp(1j*np.arange(-N//2+1,0,1)*points[i])
        else: # the number of points is odd: N=2K+1
            # K+1 coefficients with non negative index
            weights[0:(N+1)//2]=np.exp(1j*np.arange(0,(N-1)//2+1)*points[i])
            # K coefficients with negative index
            weights[(N+1)//2:]=np.exp(1j*np.arange(-(N-1)//2,0,1)*points[i])
        values[i]=np.sum(coefficients*weights)/N
    return values

**Question.** Is the Fourier interpolant of a real function always real?

**Answer.** The Fourier interpolant of a real function is not always a real function, as seen in the following numerical example with $f(x)=\cos(3x)$ and $N=6$.

In [ ]:
y=np.linspace(0,2*np.pi,1000)
N=6
plt.plot(y,np.cos(3*y),label=r'$f$')
plt.plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:np.cos(3*x),N),y).real,label=r'$\mathrm{Re}(I_6(f))$')
plt.plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:np.cos(3*x),N),y).imag,label=r'$\mathrm{Im}(I_6(f))$')
plt.legend()

**Question.** Use the written functions to plot and compare the graphs of the following functions of  $L^2([0,2\pi],\mathbb{R})$ and their respective interpolants, for several values of $N$,

* $u(x) = \cos(2x) + \sin(4x)$,
* $u(x) = \mathbb{1}_{\left[\frac{\pi}{2},\frac{3\pi}{2}\right]}(x)$,
* $u(x) = \sin (2x)\exp\left(\cos\left(8x\right)\right)$.
  
Is the [Gibbs phenomenon](https://en.wikipedia.org/wiki/Gibbs_phenomenon) observed for one of these functions?

In [ ]:
N_values=[5,10,20,40]

y=np.linspace(0,2*np.pi,1000)

fig,axs=plt.subplots(4,3,figsize=(15,15))

for i,N in enumerate(N_values):
    axs[i,0].plot(y,np.cos(2*y)+np.sin(4*y),label=r'$\cos(2x)+\sin(4x)$')
    axs[i,0].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:np.cos(2*x)+np.sin(4*x),N),y).real,label='Fourier interpolant')
    axs[i,0].set_title('N='+str(N))
    axs[i,0].legend()

    axs[i,1].plot(y,((y>0.5*np.pi) & (y<1.5*np.pi)),label=r'$\mathbb{1}_{\left[\frac{\pi}{2},\frac{3\pi}{2}\right]}(x)$')
    axs[i,1].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:((x>0.5*np.pi) & (x<1.5*np.pi)),N),y).real,label='Fourier interpolant')
    axs[i,1].set_title('N='+str(N))
    axs[i,1].legend()

    axs[i,2].plot(y,np.sin(2*y)*np.exp(np.cos(8*y)),label=r'$\sin(2x)\exp(\cos(8x))$')
    axs[i,2].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:np.sin(2*x)*np.exp(np.cos(8*x)),N),y).real,label='Fourier interpolant')
    axs[i,2].set_title('N='+str(N))
    axs[i,2].legend()

fig.tight_layout()

**Answer.** The Gibbs phenomenon is observed for the second function, which possesses discontinuities.

## Exercise 2. The Fourier interpolation derivative.

The *Fourier interpolation derivative* of a function $u$ is defined as the derivative of the interpolant $I_Nu$, that is
$$
\mathcal{D}_Nu=(I_Nu)',
$$
and, using the previous definitions, one can write, for even non-zero natural integer $N$,
$$
\mathcal{D}_Nu(x)= \sum_{k=-N/2+1}^{N/2}{ik\tilde{u}_k e^{ikx}}.
$$
As a consequence, it is easy to compute the Fourier interpolation derivative of a function once the Fourier interpolant of this function is known. 

Note that, since interpolation and differentiation do not commute, one has in general
$$
(I_nu)'\neq I_N(u'),
$$
and one can then wonder if the interpolation derivative is a good approximation to $u'$.

**Question.** Write a function which computes the coefficients of the Fourier interpolation derivative $\mathcal{D}_Nu$ of a function $u$, the set of discrete Fourier coefficients of the interpolant $I_Nu$ being given.

In [ ]:
def fourier_differentiation_coefficients(coefficients):
    N=coefficients.size
    weights=1j*np.zeros(N)
    if N%2==0: # the number of points is even
        weights[0:N//2+1]=1j*np.arange(0,N//2+1)
        weights[N//2+1:]=1j*np.arange(-N//2+1,0,1)
    else: # the number of points is odd
        weights[0:(N+1)//2]=1j*np.arange(0,(N+1)//2)
        weights[(N+1)//2:]=1j*np.arange(-(N-1)//2,0,1)
    return coefficients*weights

**Question.** Using the `np.fft.ifft` function for the inverse discrete Fourier transform, write a function which computes the values of the Fourier interpolation derivative $\mathcal{D}_Nu$ at the interpolation nodes, the discrete Fourier coefficients of $\mathcal{D}_Nu$ being given.

In [ ]:
def fourier_differentiation_values_at_nodes(differentiation_coefficients):
    return np.fft.ifft(differentiation_coefficients)

**Question.** Compare the graphs of the derivatives the following periodic functions of $L^2([0,2\pi],\mathbb{R})$ with the values at the interpolation nodes of their respective Fourier interpolation derivatives, for several values of $N$,
* $u(x) = \cos(x) + \sin(2x)$,
* $u(x) = \mathbb{1}_{\left[\frac{\pi}{2},\frac{3\pi}{2}\right]}(x)$,
* $u(x) = \sin (2x)\exp\left(\cos\left(8x\right)\right)$.

In [ ]:
N_values=[5,10,20,40]

y=np.linspace(0,2*np.pi,1000)

fig,axs=plt.subplots(4,3,figsize=(15,15))

for i,N in enumerate(N_values):
    
    x=np.linspace(0,2*np.pi,N,endpoint=False)

    axs[i,0].plot(y,-np.sin(y)+2.*np.cos(2*y),label=r'$-\sin(x)+2\cos(2x)$')
    axs[i,0].plot(x,fourier_differentiation_values_at_nodes(fourier_differentiation_coefficients(fourier_interpolant_coefficients(lambda x:np.cos(x)+np.sin(2*x),N))).real,'*',label='Fourier interpolation derivative')
    axs[i,0].set_title('N='+str(N))
    axs[i,0].legend()

    axs[i,1].plot(y,np.zeros(y.size),label=r'$\mathbb{1}_{\left[\frac{\pi}{2},\frac{3\pi}{2}\right]}(x)$')
    axs[i,1].plot(x,fourier_differentiation_values_at_nodes(fourier_differentiation_coefficients(fourier_interpolant_coefficients(lambda x:((x>0.5*np.pi) & (x<1.5*np.pi)),N))).real,'*',label='Fourier interpolation derivative')
    axs[i,1].set_title('N='+str(N))
    axs[i,1].legend()

    axs[i,2].plot(y,(2*np.cos(2*y)-8*np.sin(8*y)*np.sin(2*y))*np.exp(np.cos(8*y)),label=r'$(2\cos(2x)-8\sin(2x)\sin(8x))\exp(\cos(8x))$')
    axs[i,2].plot(x,fourier_differentiation_values_at_nodes(fourier_differentiation_coefficients(fourier_interpolant_coefficients(lambda x:np.sin(2*x)*np.exp(np.cos(8*x)),N))).real,'*',label='Fourier interpolation derivative')
    axs[i,2].set_title('N='+str(N))
    axs[i,2].legend()

fig.tight_layout()

## Exercise 3. Aliasing on an example.

Aliasing effects are caused by the fact that the functions $\{e^{i(k+Nm)x}\}_{m\in\mathbb{Z}}$ are indistinguishable on the discrete grid $\{\frac{2\pi j}{N},\ j=0,\dots,N-1\}$ (in other words, they are *aliases* of one another). The code below illustrates this phenomenon, showing that the modes $e^{ix}$ and $e^{9ix}$ both have the same representation on the grid $\{\frac{\pi j}{4},\ j=0,\dots,7\}$.

In [ ]:
x=np.linspace(0,2*np.pi,8,endpoint=False)

y=np.linspace(0,2*np.pi,1000)

fig,axs=plt.subplots(1,2,figsize=(10,8))
axs[0].plot(y,np.cos(y),label=r'$cos(x)$')
axs[0].plot(y,np.cos(9*y),label=r'$cos(9x)$')
axs[0].plot(x,np.cos(x),'o')
axs[0].set_aspect('equal','box')
axs[0].legend()

axs[1].plot(y,np.sin(y),label=r'$sin(x)$')
axs[1].plot(y,np.sin(9*y),label=r'$sin(9x)$')
axs[1].plot(x,np.sin(x),'o')
axs[1].set_aspect('equal','box')
axs[1].legend()

Let us consider the following periodic functions of $L^2([0,2\pi],\mathbb{R})$:
* $u(x)=\cos(3x)$,
* $v(x)=\sin(4x)$,
* $s(x)=u(x)v(x)$.

**Question.** What is the Fourier series associated with the function $s$? What is the minimum value of $N$ such that $I_Ns=s$?

**Answer.** One has $s(x)=\cos(3x)\sin(4x)=\frac{1}{2}\left(\sin(7x)-\sin(-x)\right)=\frac{1}{2}\left(\sin(7x)+\sin(x)\right)$ and the non-zero Fourier coefficients of the associated series are $\hat{s}_1=\hat{s}_7=-\frac{i}{4}$, $\hat{s}_{-1}=\hat{s}_{-7}=\frac{i}{4}$. As a consequence,  the interpolation of $s$ necessitates at least $N=2\times7+1=15$ points to be exact, while the interpolation of the function $u$ is exact using at least $N=2\times3+1=7$ points and that of the function $v$ is exact using at least $N=2\times4+1=9$ points.

**Question.** Plot the graphs of the functions $u$, $v$ and $s$, and of their respective Fourier interpolants $I_Nu$, $I_Nv$ and $I_Ns$, on the same figures for $N=7,9,15$. Comment.

In [ ]:
def u(x):
    return np.cos(3*x)

def v(x):
    return np.sin(4*x)

N_values=[7,9,15]

y=np.linspace(0,2*np.pi,1000)

fig,axs=plt.subplots(3,3,figsize=(15,6))
for i in range(3):
    N=N_values[i]
    axs[i,0].plot(y,u(y),label=r'$u(x)$')
    axs[i,0].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(u,N),y).real,label=r'$I_Nu(x)$')
    axs[i,0].set_aspect('equal','box')
    axs[i,0].set_title('N='+str(N))
    axs[i,0].legend()
    axs[i,1].plot(y,v(y),label=r'$v(x)$')
    axs[i,1].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(v,N),y).real,label=r'$I_Nv(x)$')
    axs[i,1].set_aspect('equal','box')
    axs[i,1].set_title('N='+str(N))
    axs[i,1].legend()
    axs[i,2].plot(y,u(y)*v(y),label=r'$s(x)$')
    axs[i,2].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:u(x)*v(x),N),y).real,label=r'$I_Ns(x)$')
    axs[i,2].set_aspect('equal','box')
    axs[i,2].set_title('N='+str(N))
    axs[i,2].legend()

fig.tight_layout()

**Answer.** The three plots confirm the conclusion in the previous answer.

In spectral methods, aliasing appears when dealing with nonlinear terms involving products, due to wave numbers appearing in the convolution of functions exceeding the fixed range of wavenumbers, equal to $\lfloor\frac{N}{2}\rfloor$. More precisely, an approach to avoid convolutions in spectral space is to use the inverse discrete Fourier transform to compute products in the physical space and then use the discrete Fourier transform to return to spectral space, the resulting scheme being called a [*pseudo-spectral method*](https://en.wikipedia.org/wiki/Pseudo-spectral_method). As seen in the example above, this product contains modes with higher wavenumbers which may not be correctly represented on the discrete grid. These are then aliased to modes with lower wavenumbers, which may introduce notable errors in the method or even result in a numerical instability.

A cure to this effect is to use a grid fine enough to correctly resolve all the modes involved in the computations, but basic techniques for aliasing removal in quadratic nonlinearities have also been proposed in the literature, one of them being the *two-thirds rule* introduced by Orszag in a [very short note](https://doi.org/10.1175/1520-0469(1971)028%3C1074:OTEOAI%3E2.0.CO;2). It works as follows.

- Assuming the $N$ gridpoints are used, the two arrays storing the spectral coefficients of the functions to be multiplied are first extended to a larger size $M=\frac{3N}{2}$ (since $N=\frac{2M}{3}$, the number of de-aliased coefficients is two-thirds of the total number of coefficients, hence the name of the technique) and padded on both sides with zeros.
- Inverse discrete Fourier transforms are performed on the resulting arrays and the product is computed in the physical space.
- The discrete Fourier transform of the product is computed and the $M-N$ discrete Fourier coefficients corresponding to higher wavenumbers are dropped by truncation, while the remaining ones yield the de-aliased coefficients after multiplication by $\frac{3}{2}$.

Let us briefly explain the rationale behind such a choice for $M$. Let
$$
\forall j\in\{0,\dots,M-1\},\ y_j=\frac{2\pi j}{M},\ u_j=\frac{1}{M}\sum_{k=-M/2+1}^{M/2}\check{u}_ke^{iky_j},\ v_j=\frac{1}{M}\sum_{k=-M/2+1}^{M/2}\check{v}_ke^{iky_j},\ s_j=u_jv_j,
$$
where
$$
\forall k\in\left\{-\frac{M}{2}+1,\dots,\frac{M}{2}\right\},\ \check{u}_k=\begin{cases}\hat{u}_k&\text{if }-\frac{N}{2}+1\leq k\leq\frac{N}{2}\\0&\text{otherwise}\end{cases}\text{ and }\check{v}_k=\begin{cases}\hat{v}_k&\text{if }-\frac{N}{2}+1\leq k\leq\frac{N}{2}\\0&\text{otherwise}\end{cases},
$$
with $\hat{u}_k$ and $\hat{v}_k$, $k=-\frac{N}{2}+1,\dots,\frac{N}{2}$, the respective Fourier coefficients of $u$ and $v$.
Let
$$
\forall k\in\left\{-\frac{M}{2}+1,\dots,\frac{M}{2}\right\},\ \check{s}_k=\sum_{j=0}^{M}s_je^{-iky_j}.
$$
Then, one has
$$
\check{s}_k=\sum_{m+n=k}\check{u}_m\check{v}_n+\sum_{m+n=k\pm M}\check{u}_m\check{v}_n,
$$
and since we are only interested in the coefficients $\check{s}_k$ for which $k$ belongs to $\left\{-\frac{N}{2}+1,\dots,\frac{N}{2}\right\}$, we choose $M$ so that the second sum on the right-hand side, the *aliasing error*, vanishes for these indices. The coefficients $\check{u}_m$ and $\check{v}_m$ being zero for $m>\frac{N}{2}+1$ or $m<-\frac{N}{2}$, we demand that, for any $m$ and $n$ in $\left\{-\frac{N}{2}+1,\dots,\frac{N}{2}\right\}$,
$$
m+n-M<-\frac{N}{2}+1\text{ if }m+n>\frac{M}{2},
$$
or
$$
m+n+M>\frac{N}{2}\text{ if }m+n<-\frac{M}{2}+1.
$$
The worst-case condition is obtained for the choice $m=n=\frac{N}{2}$ in the first of these two cases, which gives
$$
M>\frac{3N}{2}-1.
$$
With $M$ chosen in this way, we have obtained the de-aliased coefficients of $s$, that is
$$
\forall k\in\left\{-\frac{N}{2}+1,\dots,\frac{N}{2}\right\},\ \check{s}_k=\hat{s}_k,
$$
where $\hat{s}_k=\sum_{m+n=k}\hat{u}_m\hat{v}_n$.

It has to be stressed that this method only helps to suppress aliasing errors and does not increase the resolution of the numerical solution. Note that the technique can be adapted to the case of the product of $p$ functions by setting $M=\frac{p+1}{2}N$.

**Question.** Implement the two-thirds rule algorithm computing the interpolation coefficients of the product of two functions, the respective sets of interpolation coefficients of the functions being given. Test it on the previous example.

In [ ]:
def dealiasing(u_tilde,v_tilde):
    # de-aliasing by the 2/3-rule
    N=u_tilde.size
    # zero-padding to compute the nonlinear product in physical space on a grid with (3/2)N points
    M=3*N//2
    if N%2==0: # the number of points is even
        u_tilde_pad=np.concatenate((u_tilde[0:N//2+1],1j*np.zeros(M-N),u_tilde[N//2+1:]))
        v_tilde_pad=np.concatenate((v_tilde[0:N//2+1],1j*np.zeros(M-N),v_tilde[N//2+1:]))
    else: # the number of points is odd
        u_tilde_pad=np.concatenate((u_tilde[0:(N+1)//2],1j*np.zeros(M-N),u_tilde[(N+1)//2:]))
        v_tilde_pad=np.concatenate((v_tilde[0:(N+1)//2],1j*np.zeros(M-N),v_tilde[(N+1)//2:]))
    # inverse discrete Fourier transforms
    u_pad=np.fft.ifft(u_tilde_pad)
    v_pad=np.fft.ifft(v_tilde_pad)
    # product
    s_pad=u_pad*v_pad
    # discrete Fourier transform
    s_tilde_pad=np.fft.fft(s_pad)
    # truncation
    s_tilde=1j*np.zeros(N)
    if N%2==0: # the number of points is even
        s_tilde[0:N//2+1]=1.5*s_tilde_pad[0:N//2+1]
        s_tilde[N//2+1:]=1.5*s_tilde_pad[M-N//2+1:]
    else:
        s_tilde[0:(N+1)//2]=1.5*s_tilde_pad[0:(N-1)//2+1]
        s_tilde[(N+1)//2:]=1.5*s_tilde_pad[M-(N-1)//2:]
    return s_tilde

# test on the previous example
fig,axs=plt.subplots(1,3,figsize=(18,5))
for i in range(3):
    N=N_values[i]
    
    dealiased_coefficients=dealiasing(fourier_interpolant_coefficients(u,N),fourier_interpolant_coefficients(v,N))
    axs[i].plot(y,u(y)*v(y),label=r'$s(x)$')
    axs[i].plot(y,evaluate_fourier_interpolant(fourier_interpolant_coefficients(lambda x:u(x)*v(x),N),y).real,label=r'$I_Ns(x)$')
    axs[i].plot(y,evaluate_fourier_interpolant(dealiased_coefficients,y).real,label=r'de-aliased $I_Ns(x)$')
    axs[i].set_aspect('equal','box')
    axs[i].set_title('N='+str(N))
    axs[i].legend()

fig.tight_layout()